In [1]:
import os
import subprocess
import numpy as np
import pandas as pd
import random
import mplcursors
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import font_manager

In [2]:
def run_cpp_program(noise, nPose):
    result = subprocess.run(['../build/MyGTSAM', f'{noise}', f'{nPose}'], 
                            capture_output=True, text=True)

    if result.returncode != 0:
        print(f"Error running C++ program: {result.stderr}")
        return np.inf

    # print(f"C++ program output:\n{result.stdout}")

    return result.stdout.strip().split('\n')

In [3]:
content = pd.read_csv('pose_GT_8.csv', header=None)
pose_GT = content.values
pose_GT = np.delete(pose_GT, 2, axis=1)

content = pd.read_csv("/home/ali/Github/Kamal_Dataset/seq8/encoder/encoder.csv", header=None)
Data = content.values
delta = Data[:,0]
tsta = Data[:,1]
brk = Data[:,2]

content = pd.read_csv('initial_estimates_8.csv', header=None)
initialEst = content.values

nPose = len(initialEst)-8

cable_GT = initialEst[nPose:nPose+4,0].copy()
anchor_GT = initialEst[nPose+4:,:].copy()

# Perturbation Results
## Sequence 8:
    Anchor Points: 0.2
    Cable Length Offsets: 0.1

In [4]:
result = []
pose = []
anchor_STD = 0.3
cable_STD = 0.0
cable_prior_sigma = 0.1
file_path = 'perturb.csv'
file_exists = os.path.isfile(file_path)
for i in range(250):
    initialEst[nPose,:] = np.random.normal(cable_GT[0], cable_STD/(2*np.sqrt(2)), size=2)  # m0
    initialEst[nPose+1,:] = np.random.normal(cable_GT[1], cable_STD/(2*np.sqrt(2)), size=2)  # n0
    initialEst[nPose+2,:] = np.random.normal(cable_GT[2], cable_STD/(2*np.sqrt(2)), size=2)  # k0
    initialEst[nPose+3,:] = np.random.normal(cable_GT[3], cable_STD/(2*np.sqrt(2)), size=2)  # h0

    initialEst[nPose+4,:] = np.random.normal(anchor_GT[0], anchor_STD/(2*np.sqrt(2)), size=2)    # Anchor 1
    initialEst[nPose+5,:] = np.random.normal(anchor_GT[1], anchor_STD/(2*np.sqrt(2)), size=2)    # Anchor 2
    initialEst[nPose+6,:] = np.random.normal(anchor_GT[2], anchor_STD/(2*np.sqrt(2)), size=2)    # Anchor 3
    initialEst[nPose+7,:] = np.random.normal(anchor_GT[3], anchor_STD/(2*np.sqrt(2)), size=2)    # Anchor 4
    
    initialEst_pd = pd.DataFrame(initialEst)
    initialEst_pd.to_csv('../dataset/estimates_8.csv', header=False, index=False)
    Opt = list(map(float, run_cpp_program(cable_prior_sigma, 31)))
    values = np.concatenate((np.array((initialEst[nPose-1,0], initialEst[nPose,0], initialEst[nPose+1,0], initialEst[nPose+2,0], initialEst[nPose+3,0])),
                             initialEst[nPose+4], initialEst[nPose+5], initialEst[nPose+6], initialEst[nPose+7], Opt[:15]))
    result.append(values)
    pose.append(Opt[13:])

result = np.array(result)
pose = np.array(pose)
result_pd = pd.DataFrame(result)
result_pd.columns = ['s0', 'm00', 'n00', 'k00', 'h00', 'A10_x', 'A10_y', 'A20_x', 'A20_y', 'A30_x', 'A30_y', 'A40_x', 'A40_y',
                     's', 'm0', 'n0', 'k0', 'h0', 'A1_x', 'A1_y', 'A2_x', 'A2_y', 'A3_x', 'A3_y', 'A4_x', 'A4_y', 'X0_x', 'X0_y']
# result_pd.to_csv(file_path, mode='a', index=False, header=not file_exists)

In [5]:
rmse_m0 = np.sqrt(np.mean((result_pd['m0'] - cable_GT[0]) ** 2))
rmse_n0 = np.sqrt(np.mean((result_pd['n0'] - cable_GT[1]) ** 2))
rmse_k0 = np.sqrt(np.mean((result_pd['k0'] - cable_GT[2]) ** 2))
rmse_h0 = np.sqrt(np.mean((result_pd['h0'] - cable_GT[3]) ** 2))

rmse_a1 = np.sqrt(np.mean((result_pd['A1_x'] - anchor_GT[0,0])**2 + (result_pd['A1_y'] - anchor_GT[0,1])**2))
rmse_a2 = np.sqrt(np.mean((result_pd['A2_x'] - anchor_GT[1,0])**2 + (result_pd['A2_y'] - anchor_GT[1,1])**2))
rmse_a3 = np.sqrt(np.mean((result_pd['A3_x'] - anchor_GT[2,0])**2 + (result_pd['A3_y'] - anchor_GT[2,1])**2))
rmse_a4 = np.sqrt(np.mean((result_pd['A4_x'] - anchor_GT[3,0])**2 + (result_pd['A4_y'] - anchor_GT[3,1])**2))
rmse_x0 = np.sqrt(np.mean((result_pd['X0_x'] - 0.605)**2 + (result_pd['X0_y'] - 1.9+0.805)**2))

rmse_pose = []
for i in range(len(pose)):
    p = (pose[i]).reshape(-1,2)
    pose_GT += p[0] - pose_GT[0]
    rmse_pose.append(np.sqrt(np.mean((pose_GT[:len(p),0] - p[:,0])**2 + (pose_GT[:len(p),1] - p[:,1])**2)))

rmse_pose = np.mean(rmse_pose)

print("RMSE m0:",rmse_m0)
print("RMSE n0:",rmse_n0)
print("RMSE k0:",rmse_k0)
print("RMSE h0:",rmse_h0)

print("\nRMSE A1:",rmse_a1)
print("RMSE A2:",rmse_a2)
print("RMSE A3:",rmse_a3)
print("RMSE A4:",rmse_a4)
print("RMSE X0:",rmse_x0)

print("\nRMSE Pose:", rmse_pose)

RMSE m0: 0.038283852025050955
RMSE n0: 0.03430116802011268
RMSE k0: 0.025349999999999984
RMSE h0: 0.017388560354439937

RMSE A1: 0.04481109361723641
RMSE A2: 0.04114314083295043
RMSE A3: 0.05687185444821575
RMSE A4: 0.02749267940780425
RMSE X0: 0.00908614686762216

RMSE Pose: 0.0029794588157806157


In [6]:
s1 = np.mean(np.sqrt((result_pd['A1_x'] - result_pd['A3_x'])**2 + (result_pd['A1_y'] - result_pd['A3_y'])**2))
s2 = np.mean(np.sqrt((result_pd['A1_x'] - result_pd['A2_x'])**2 + (result_pd['A1_y'] - result_pd['A2_y'])**2))
s3 = np.mean(np.sqrt((result_pd['A2_x'] - result_pd['A4_x'])**2 + (result_pd['A2_y'] - result_pd['A4_y'])**2))
s4 = np.mean(np.sqrt((result_pd['A3_x'] - result_pd['A4_x'])**2 + (result_pd['A3_y'] - result_pd['A4_y'])**2))

print(s1)
print(s2)
print(s3)
print(s4)

1.8590440742502028
1.151206634486543
1.8529507023246585
1.1556900396290957


In [7]:
a = 1.20
b = 1.90

r_Delta = 0.025
r_TSTA = 0.035
r_Break = 0.035
X0 = [0.605, b - 0.805]

A1 = [0, b]
A2 = [a, b]
A3 = [0, 0]
A4 = [a, 0]

m0 = np.sqrt((X0[0] - A1[0])**2 + (X0[1] - A1[1])**2)
n0 = np.sqrt((X0[0] - A2[0])**2 + (X0[1] - A2[1])**2)
k0 = np.sqrt((X0[0] - A3[0])**2 + (X0[1] - A3[1])**2)
h0 = np.sqrt((X0[0] - A4[0])**2 + (X0[1] - A4[1])**2)

l0_Delta = m0 + n0
l0_TSTA = m0 + 2*k0
l0_Break = n0 + 2*h0

l_Delta = r_Delta*delta + l0_Delta
l_TSTA = r_TSTA*tsta + l0_TSTA
l_Break = r_Break*brk + l0_Break

den = 2*l_Delta + 0.5*(l_TSTA + l_Break - l_Delta)
num = l_Delta**2 - 0.25*(l_Break**2 + l_Delta**2 - 2*l_Delta*l_Break - l_TSTA**2)
m = num/den
n = l_Delta - m
k = 0.5*(l_TSTA - m)
h = 0.5*(l_Break - n)

l = [m, n, k, h]

x_kin = (m**2 - n**2 + a**2)/(2*a)
y_kin = b - np.sqrt(m**2 - x_kin**2)

pose_GT[:,0] += x_kin[0] - pose_GT[0,0]
pose_GT[:,1] += y_kin[0] - pose_GT[0,1]
rmse_pose_nom = []
for i in range(len(pose)):
    rmse_pose_nom.append(np.sqrt(np.mean((pose_GT[:,0] - x_kin)**2 + (pose_GT[:,1] - y_kin)**2)))

rmse_pose_nom = np.mean(rmse_pose_nom)
print(rmse_pose_nom)

0.0037786621630821146


In [8]:
# print((rmse_pose_nom - rmse_pose)/rmse_pose_nom*100)

In [24]:
plt.figure()
%matplotlib qt

plt.scatter(result_pd['A10_x'], result_pd['A10_y'], color="tab:cyan", label="Anchor 1 Initial Values")
plt.scatter(result_pd['A20_x'], result_pd['A20_y'], color="tab:pink", label="Anchor 2 Initial Values")
plt.scatter(result_pd['A30_x'], result_pd['A30_y'], color="gold", label="Anchor 3 Initial Values")
plt.scatter(result_pd['A40_x'], result_pd['A40_y'], color="limegreen", label="Anchor 4 Initial Values")

mean = np.mean(np.column_stack((result_pd['A1_x'], result_pd['A1_y'])), axis=0)
plt.scatter(result_pd['A1_x'], result_pd['A1_y'], label=f'Anchor 1 Optimization Result', color="tab:blue")
# plt.scatter(result_pd['A1_x'], result_pd['A1_y'], label=f'Anchor 1 Optimization Result: ({mean[0]:.3f}, {mean[1]:.3f})', color="tab:blue")

mean = np.mean(np.column_stack((result_pd['A2_x'], result_pd['A2_y'])), axis=0)
plt.scatter(result_pd['A2_x'], result_pd['A2_y'], label=f'Anchor 2 Optimization Result', color="purple")
# plt.scatter(result_pd['A2_x'], result_pd['A2_y'], label=f'Anchor 2 Optimization Result: ({mean[0]:.3f}, {mean[1]:.3f})', color="tab:purple")

mean = np.mean(np.column_stack((result_pd['A3_x'], result_pd['A3_y'])), axis=0)
plt.scatter(result_pd['A3_x'], result_pd['A3_y'], label=f'Anchor 3 Optimization Result', color="tab:orange")
# plt.scatter(result_pd['A3_x'], result_pd['A3_y'], label=f'Anchor 3 Optimization Result: ({mean[0]:.3f}, {mean[1]:.3f})', color="tab:orange")

mean = np.mean(np.column_stack((result_pd['A4_x'], result_pd['A4_y'])), axis=0)
plt.scatter(result_pd['A4_x'], result_pd['A4_y'], label=f'Anchor 4 Optimization Result', color="green")
# plt.scatter(result_pd['A4_x'], result_pd['A4_y'], label=f'Anchor 4 Optimization Result: ({mean[0]:.3f}, {mean[1]:.3f})', color="green")

# mean = np.mean(np.column_stack((result_pd['X0_x'], result_pd['X0_y'])), axis=0)
# plt.scatter(result_pd['X0_x'], result_pd['X0_y'], label=f'X0 Optimization Result: ({mean[0]:.3f}, {mean[1]:.3f})', color="tab:red")

plt.axis('equal')
csfont = {'fontname':'Times New Roman', 'fontsize': 22}
font = font_manager.FontProperties(family='Times New Roman',
                                   style='normal', size=18)
# plt.legend(prop=font)
plt.xlabel("x (m)", **csfont)
plt.ylabel("y (m)", **csfont)

plt.tick_params(axis='both', which='major', labelsize=18, labelfontfamily='Times New Roman')
mplcursors.cursor(multiple=True)

# plt.subplots()

# plt.subplot(4,1,1)
# plt.hist(result_pd['m0'], bins=100, edgecolor='black')
# mean = np.mean(result_pd['m0'])
# std_dev = np.std(result_pd['m0'])
# plt.legend([f'Without Perturbation: {cable_GT[0]}\nMean: {mean:.3f}\nRMSE: {rmse_m0:.3f}'])
# plt.ylabel('$m_0$')

# plt.subplot(4,1,2)
# plt.hist(result_pd['n0'], bins=100, edgecolor='black')
# mean = np.mean(result_pd['n0'])
# std_dev = np.std(result_pd['n0'])
# plt.legend([f'Without Perturbation: {cable_GT[1]}\nMean: {mean:.3f}\nRMSE: {rmse_n0:.3f}'])
# plt.ylabel('$n_0$')

# plt.subplot(4,1,3)
# plt.hist(result_pd['k0'], bins=100, edgecolor='black')
# mean = np.mean(result_pd['k0'])
# std_dev = np.std(result_pd['k0'])
# plt.legend([f'Without Perturbation: {cable_GT[2]}\nMean: {mean:.3f}\nRMSE: {rmse_k0:.3f}'])
# plt.ylabel('$k_0$')

# plt.subplot(4,1,4)
# plt.hist(result_pd['h0'], bins=100, edgecolor='black')
# mean = np.mean(result_pd['h0'])
# std_dev = np.std(result_pd['h0'])
# plt.legend([f'Without Perturbation: {cable_GT[3]}\nMean: {mean:.3f}\nRMSE: {rmse_h0:.3f}'])
# plt.ylabel('$h_0$')
# plt.xlabel('Length (m)')
# plt.suptitle(f'$Perturbation={cable_STD}$')

# mplcursors.cursor(multiple=True)